In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler


In [ ]:
df_path = '/content/drive/MyDrive/output.csv'
df = pd.read_csv(df_path)

In [ ]:
df.head(2)

,Date,stid,Daily Solar Energy,new_distances,min_dist_node,join,tcolc_eatm_0,ulwrf_tatm_0,dlwrf_sfc_0,tmp_sfc_0,...,dswrf_sfc_0,tmax_2m_0,tmin_2m_0,pwat_eatm_0,uswrf_sfc_0,spfh_2m_0,ulwrf_sfc_0,tmp_2m_0,apcp_sfc_0,pres_msl_0
0,1994-01-01,ACME,12384900,"{(33.0, 257.0): 501.30264193302355, (33.0, 258...","(35.0, 262.0)","19940101(35.0, 262.0)",0.002298,254.38380,258.64188,282.54855,...,219.09091,284.39706,280.26060,8.086300,47.745450,0.003322,364.3198,282.35430,0.000000,102023.78
1,1994-01-02,ACME,11908500,"{(33.0, 257.0): 501.30264193302355, (33.0, 258...","(35.0, 262.0)","19940102(35.0, 262.0)",0.069067,231.38722,279.48965,282.33215,...,166.90909,283.79266,279.78726,11.476501,32.618183,0.003798,358.0753,282.07254,0.031273,101307.50


In [ ]:
df.head()

,Date,stid,Daily Solar Energy,new_distances,min_dist_node,join,tcolc_eatm_0,ulwrf_tatm_0,dlwrf_sfc_0,tmp_sfc_0,...,dswrf_sfc_0,tmax_2m_0,tmin_2m_0,pwat_eatm_0,uswrf_sfc_0,spfh_2m_0,ulwrf_sfc_0,tmp_2m_0,apcp_sfc_0,pres_msl_0
0,1994-01-01,ACME,12384900,"{(33.0, 257.0): 501.30264193302355, (33.0, 258...","(35.0, 262.0)","19940101(35.0, 262.0)",0.002298,254.38380,258.64188,282.54855,...,219.09091,284.39706,280.26060,8.086300,47.745450,0.003322,364.31980,282.35430,0.000000,102023.780
1,1994-01-02,ACME,11908500,"{(33.0, 257.0): 501.30264193302355, (33.0, 258...","(35.0, 262.0)","19940102(35.0, 262.0)",0.069067,231.38722,279.48965,282.33215,...,166.90909,283.79266,279.78726,11.476501,32.618183,0.003798,358.07530,282.07254,0.031273,101307.500
2,1994-01-03,ACME,12470700,"{(33.0, 257.0): 501.30264193302355, (33.0, 258...","(35.0, 262.0)","19940103(35.0, 262.0)",0.001782,241.27985,227.04659,277.42630,...,225.32729,277.42557,273.05588,6.755241,49.072730,0.002511,335.16843,275.72464,0.000000,101984.920
3,1994-01-04,ACME,12725400,"{(33.0, 257.0): 501.30264193302355, (33.0, 258...","(35.0, 262.0)","19940104(35.0, 262.0)",0.003251,244.66994,238.26636,277.47464,...,226.21817,278.94736,275.17150,5.734901,49.527270,0.002601,342.49518,276.33093,0.000909,102571.125
4,1994-01-05,ACME,10894800,"{(33.0, 257.0): 501.30264193302355, (33.0, 258...","(35.0, 262.0)","19940105(35.0, 262.0)",0.025482,242.21690,258.61813,282.66803,...,219.45454,283.42203,278.55490,9.284234,46.636364,0.003136,353.97708,282.82280,0.000000,100769.320


In [ ]:
# Function to prepare data for a single station
def prepare_station_data(station_data, seq_length=15):
    X, y = [], []
    for i in range(len(station_data) - seq_length):
        X.append(station_data.iloc[i:i + seq_length, 3:19].values)
        y.append(station_data.iloc[i + seq_length, 2])
    X, y = np.array(X), np.array(y)

    # Normalize input features
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

    return X, y


In [ ]:
# Get a list of unique station identifiers
stations = df['stid'].unique()


In [ ]:
stations

array(['ACME', 'ADAX', 'ALTU', 'APAC', 'ARNE', 'BEAV', 'BESS', 'BIXB',
       'BLAC', 'BOIS', 'BOWL', 'BREC', 'BRIS', 'BUFF', 'BURB', 'BURN',
       'BUTL', 'BYAR', 'CAMA', 'CENT', 'CHAN', 'CHER', 'CHEY', 'CHIC',
       'CLAY', 'CLOU', 'COOK', 'COPA', 'DURA', 'ELRE', 'ERIC', 'EUFA',
       'FAIR', 'FORA', 'FREE', 'FTCB', 'GOOD', 'GUTH', 'HASK', 'HINT',
       'HOBA', 'HOLL', 'HOOK', 'HUGO', 'IDAB', 'JAYX', 'KENT', 'KETC',
       'LAHO', 'LANE', 'MADI', 'MANG', 'MARE', 'MAYR', 'MCAL', 'MEDF',
       'MEDI', 'MIAM', 'MINC', 'MTHE', 'NEWK', 'NINN', 'NOWA', 'OILT',
       'OKEM', 'OKMU', 'PAUL', 'PAWN', 'PERK', 'PRYO', 'PUTN', 'REDR',
       'RETR', 'RING', 'SALL', 'SEIL', 'SHAW', 'SKIA', 'SLAP', 'SPEN',
       'STIG', 'STIL', 'STUA', 'SULP', 'TAHL', 'TALI', 'TIPT', 'TISH',
       'VINI', 'WASH', 'WATO', 'WAUR', 'WEAT', 'WEST', 'WILB', 'WIST',
       'WOOD', 'WYNO'], dtype=object)

In [ ]:
station_data = df[df['stid'] == 'ACME']

In [ ]:
data = station_data.copy()

In [ ]:
data.drop(['new_distances', 'min_dist_node', 'join'], axis=1, inplace=True)

In [ ]:
data.head()

,Date,stid,Daily Solar Energy,tcolc_eatm_0,ulwrf_tatm_0,dlwrf_sfc_0,tmp_sfc_0,tcdc_eatm_0,dswrf_sfc_0,tmax_2m_0,tmin_2m_0,pwat_eatm_0,uswrf_sfc_0,spfh_2m_0,ulwrf_sfc_0,tmp_2m_0,apcp_sfc_0,pres_msl_0
0,1994-01-01,ACME,12384900,0.002298,254.38380,258.64188,282.54855,0.001273,219.09091,284.39706,280.26060,8.086300,47.745450,0.003322,364.31980,282.35430,0.000000,102023.780
1,1994-01-02,ACME,11908500,0.069067,231.38722,279.48965,282.33215,0.069091,166.90909,283.79266,279.78726,11.476501,32.618183,0.003798,358.07530,282.07254,0.031273,101307.500
2,1994-01-03,ACME,12470700,0.001782,241.27985,227.04659,277.42630,0.000909,225.32729,277.42557,273.05588,6.755241,49.072730,0.002511,335.16843,275.72464,0.000000,101984.920
3,1994-01-04,ACME,12725400,0.003251,244.66994,238.26636,277.47464,0.002727,226.21817,278.94736,275.17150,5.734901,49.527270,0.002601,342.49518,276.33093,0.000909,102571.125
4,1994-01-05,ACME,10894800,0.025482,242.21690,258.61813,282.66803,0.024364,219.45454,283.42203,278.55490,9.284234,46.636364,0.003136,353.97708,282.82280,0.000000,100769.320


In [ ]:
data.tail()

,Date,stid,Daily Solar Energy,tcolc_eatm_0,ulwrf_tatm_0,dlwrf_sfc_0,tmp_sfc_0,tcdc_eatm_0,dswrf_sfc_0,tmax_2m_0,tmin_2m_0,pwat_eatm_0,uswrf_sfc_0,spfh_2m_0,ulwrf_sfc_0,tmp_2m_0,apcp_sfc_0,pres_msl_0
5108,2007-12-27,ACME,1408500,0.008116,239.14300,232.15019,274.65700,0.007091,212.72728,274.60257,271.48920,8.197086,59.636364,0.003368,320.14722,273.69574,0.002364,101556.650
5109,2007-12-28,ACME,10060800,0.017351,235.86494,226.88014,272.39377,0.017091,223.09091,273.36935,270.51187,6.018736,90.436370,0.002995,313.22960,271.98865,0.009273,102168.800
5110,2007-12-29,ACME,11388000,0.049416,223.78392,232.12690,272.60520,0.048727,209.81818,274.08612,269.55624,8.348250,82.581820,0.002958,309.80417,272.73972,0.000364,102100.625
5111,2007-12-30,ACME,12441000,0.000593,242.54750,224.02965,273.83890,0.000000,228.36363,276.13943,271.73752,4.311398,85.890910,0.003054,316.94860,274.70676,0.000000,101760.250
5112,2007-12-31,ACME,12450300,0.000580,246.58359,230.03691,275.28226,0.000000,227.27272,277.49857,274.59988,4.845500,79.218185,0.003058,326.44357,276.04416,0.000000,102298.630


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
# Split data into train and test sets based on date (80% train, 20% test)
train_data_end_date = data['Date'].quantile(0.8)
train_data = data[data['Date'] <= train_data_end_date]
test_data = data[data['Date'] > train_data_end_date]
print(train_data_end_date)

2005-03-13 14:24:00


In [ ]:
 # train and test data
X_train, y_train = prepare_station_data(train_data)
X_test, y_test = prepare_station_data(test_data)
print(X_train.shape)

(4075, 15, 15)


In [ ]:
print(train_data.shape)

(4090, 18)


In [43]:
model = Sequential([
    Conv1D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1)
])

In [44]:
# Compilation
# learning rate
learning_rate = 0.01

# an instance of the Adam optimizer with the specified learning rate
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the custom optimizer
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [45]:
early_stop = EarlyStopping(monitor='loss', patience=5)


In [46]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), verbose=1)

Epoch 1/100
255/255 [==============================] - 2s 4ms/step - loss: 132808030814208.0000 - mae: 8803629.0000 - val_loss: 33820082438144.0000 - val_mae: 4460741.0000
Epoch 2/100
255/255 [==============================] - 1s 4ms/step - loss: 43575769300992.0000 - mae: 5314867.5000 - val_loss: 32548015046656.0000 - val_mae: 4446880.0000
Epoch 3/100
255/255 [==============================] - 1s 4ms/step - loss: 42303808864256.0000 - mae: 5210634.0000 - val_loss: 32003091070976.0000 - val_mae: 4433127.0000
Epoch 4/100
255/255 [==============================] - 1s 4ms/step - loss: 41313122648064.0000 - mae: 5146628.5000 - val_loss: 33327262203904.0000 - val_mae: 4333927.0000
Epoch 5/100
255/255 [==============================] - 1s 4ms/step - loss: 41381015846912.0000 - mae: 5144796.5000 - val_loss: 30905934544896.0000 - val_mae: 4384545.5000
Epoch 6/100
255/255 [==============================] - 1s 4ms/step - loss: 40442951368704.0000 - mae: 5123237.0000 - val_loss: 31868879634432.00

In [47]:
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

In [48]:
import math

In [49]:
print(f"Station ACME - Test Loss(RMSE): {math.sqrt(test_loss)}, Test MAE: {test_mae}")


Station ACME - Test Loss(RMSE): 5382291.285655952, Test MAE: 4058676.75
